# RAG with LangChain, Ollama, and FAISS Vector Store

## PDF Dataset: 
https://github.com/aydiegithub/rag-system-ollama/tree/dac3b4563a66b8b11962aaa08349ba5138be396a/rag-dataset-main

![Document Ingestion](flowcharts/Flowchart.png)

In [1]:
# pip install -U langchain-community faiss-cpu langchain-huggingface pymupdf tiktoken langchain-ollama python-dotenv

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import os
from dotenv import load_dotenv

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
load_dotenv()

True

## Document loader

In [4]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("rag-dataset-main/machine-learning/MACHINE LEARNING(R17A0534).pdf")

docs = loader.load()

In [5]:
doc = docs[10]

In [6]:
print(doc.page_content)

6 
 
covers less distance physically than by train because a plane is unrestricted. Similarly, in chess, the 
concept of distance depends on the piece used – for example, a Bishop can move diagonally.   Thus, 
depending on the entity and the mode of travel, the concept of distance can be experienced differently. 
The distance metrics commonly used are Euclidean, Minkowski, Manhattan, and Mahalanobis. 
 
 
Distance is applied through the concept of neighbours and exemplars. Neighbours are points in 
proximity with respect to the distance measure expressed through exemplars. Exemplars are 
either centroids that ﬁnd a centre of mass according to a chosen distance metric or medoids that ﬁnd 
the most centrally located data point. The most commonly used centroid is the arithmetic mean, which 
minimises squared Euclidean distance to all other points. 
 
Notes: 
 
The centroid represents the geometric centre of a plane figure, i.e., the arithmetic mean 
position of all the points in the figu

## Load all the pdfs

In [7]:
import os

pdfs = []

for root, dirs, files in os.walk('rag-dataset-main'):
    # print(root, dirs, files)
    for file in files:
        if file.endswith('.pdf'):
            pdfs.append(os.path.join(root, file))

pdfs

['rag-dataset-main/machine-learning/CSIT_(R22)_3-2_MACHINE LEARNING_DIGITAL NOTES.pdf',
 'rag-dataset-main/machine-learning/NotesOnMachineLearningForBTech-1.pdf',
 'rag-dataset-main/machine-learning/ML_notes_22.pdf',
 'rag-dataset-main/machine-learning/COS324_Course_Notes.pdf',
 'rag-dataset-main/machine-learning/2505.03861v1.pdf',
 'rag-dataset-main/machine-learning/MACHINE LEARNING(R17A0534).pdf']

In [8]:
docs = []

for pdf in pdfs:
    loader = PyMuPDFLoader(pdf)
    pages = loader.load()
    
    docs.extend(pages)

In [9]:
len(docs) # number of pages in document

1060

## Document Chuncking

In [10]:
# pip install -qU langchain-text-splitters

In [23]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)

chunks = text_splitter.split_documents(docs)

In [24]:
len(docs), len(chunks)

(1060, 2440)

In [30]:
print(chunks[100].page_content)

CSIT DEPT-R22-MACHINE LEARNING 
48 
 
The multi-layer perceptron model is also known as the Backpropagation algorithm, which 
executes in two stages as follows: 
 
Forward Stage: Activation functions start from the input layer in the forward stage 
and terminate on the output layer. 
 
Backward Stage: In the backward stage, weight and bias values are modified as per 
the model's requirement. In this stage, the error between actual output and demanded 
originated backward on the output layer and ended on the input layer. 
Hence, a multi-layered perceptron model has considered as multiple artificial neural networks 
having various layers in which activation function does not remain linear, similar to a single 
layer perceptron model. Instead of linear, activation function can be executed as sigmoid, 
TanH, ReLU, etc., for deployment. 
A multi-layer perceptron model has greater processing power and can process linear and non-


In [33]:
len(chunks[100].page_content)

938

In [34]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o-mini")

In [39]:
len(encoding.encode(chunks[2].page_content))

164

In [40]:
len(encoding.encode(docs[2].page_content))

314

## Document Vector Embedding

In [45]:
from langchain_ollama import OllamaEmbeddings

import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore # used to load the vectors in the ram

In [51]:
embeddings = OllamaEmbeddings(model = 'nomic-embed-text', base_url = "http://localhost:11434")

In [57]:
single_vector = embeddings.embed_query("Hello there I am Aydie.")

In [60]:
len(single_vector), single_vector[:10]

(768,
 [-0.0056480016,
  0.012251444,
  -0.1503848,
  -0.051890645,
  0.02039635,
  0.020038703,
  -0.015681038,
  -0.038975507,
  -0.020349437,
  0.017903127])

In [63]:
index = faiss.IndexFlatL2(len(single_vector))
index, index.ntotal, index.d

(<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x135bad4d0> >,
 0,
 768)

In [64]:
vector_store = FAISS(
    embedding_function = embeddings,
    index = index,
    docstore = InMemoryDocstore(),
    index_to_docstore_id = {}
)

In [65]:
vector_store

In [66]:
docs

[Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2024-12-06T07:02:22+00:00', 'source': 'rag-dataset-main/machine-learning/CSIT_(R22)_3-2_MACHINE LEARNING_DIGITAL NOTES.pdf', 'file_path': 'rag-dataset-main/machine-learning/CSIT_(R22)_3-2_MACHINE LEARNING_DIGITAL NOTES.pdf', 'total_pages': 120, 'format': 'PDF 1.5', 'title': '', 'author': 'MRCETIT', 'subject': '', 'keywords': '', 'moddate': '2024-12-06T07:02:22+00:00', 'trapped': '', 'modDate': 'D:20241206070222Z', 'creationDate': "D:20241206070222+00'00'", 'page': 0}, page_content='DIGITAL NOTES \nOF \nMACHINE LEARNING \n[R22A6602] \n \n                            B. TECH III YEAR - II SEM \n(2024-2025) \n \n \n \n \n \n \n   PREPARED BY            \n                                           P.HARIKRISHNA \n \n             DEPARTMENT OF COMPUTER SCIENCE & INFORMATION TECHNOLOGY \n \n           MALLA REDDY COLLEGE OF ENGINEERING & TECHNOLOGY \n(Autonomous Institution – UGC, Govt. of 

In [68]:
ids = vector_store.add_documents(documents = chunks)

In [71]:
ids[:10], len(ids)

(['f1de637a-c06c-47b6-b77e-a8ebabec940c',
  'a3013ca1-a39b-4e0e-b031-bd1f66cf1bd0',
  'c18c9f8b-198e-42cb-9c87-f982ecc3dcf0',
  '1baa2d99-f6b7-4de7-8fe5-b8c42dbe387c',
  'e08b49f5-b476-4e15-8b98-25170a3d40df',
  '6ac379ba-dc6b-41e3-ac10-d60408b6b25b',
  '59a89ebc-9e7c-4027-b179-f45dfdcf9b47',
  '52c81183-560f-4f0e-9f6f-eee23adf6346',
  'f8fdb64b-a9f7-4ecb-abd2-e53a0e262eef',
  '70373b9c-1505-412c-aaac-c9ad030969af'],
 2440)

In [74]:
vector_store.index_to_docstore_id

{0: 'd58eca07-3e59-4672-bd2e-2c38e616d948',
 1: '38afa4f9-a8b8-44de-8f0e-bbac52d71490',
 2: '77d0580c-0576-49ab-b8a4-77ea162abb1e',
 3: '6b4142d1-f1fb-4e9f-8e08-30a7cb5e04c3',
 4: '5f81a121-5dba-4b42-b885-2876ca5ace30',
 5: 'ab9f21a3-f874-475d-adfb-f02a3e1af660',
 6: '1cb3b16c-4837-4b5b-88a0-1eeac6811a3c',
 7: '0cb8c823-afcb-49b9-8484-f0ef7f32b4fc',
 8: '26dad152-ed92-4a0c-a4a3-cbbcfc06feb2',
 9: 'b1bfb554-b09d-493d-a729-5dc335da6d23',
 10: '1edb36b3-e51d-4923-8484-a610fd53b9d4',
 11: 'c5921a0f-5ebe-4e28-a0e0-37c987fae66c',
 12: '035249e6-c482-40ce-a67a-8d325bd05317',
 13: '0275d68e-4b9c-4fac-aba1-167a71bbc22d',
 14: '9ff4568d-7aa8-4a94-b547-92b8ba2feda6',
 15: 'c6732cc2-5e65-4012-a09a-35a568b9eee1',
 16: 'd7607d5a-6f9b-4591-9d41-0c87ed42bb9e',
 17: 'baf2a2ae-1da8-493a-b20a-0706a56159a0',
 18: '24107200-d3b4-4537-a0de-75de3ac73ae2',
 19: 'cb4722fc-d7b5-42a7-8bb1-57675e6a4b1c',
 20: 'e232fcb1-879c-4517-8bc2-9a722945616f',
 21: '5ba0ff87-b24d-485f-a926-5e9c11b3ae0b',
 22: 'a2866a25-d516-

In [75]:
vector_store.save_local('machine_learning_vdb')

In [76]:
new_vec_store = FAISS.load_local('machine_learning_vdb', embeddings = embeddings, allow_dangerous_deserialization = True)